# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [111]:
# import the requests library (1 line)
import requests 

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url='https://country-leaders.onrender.com/status/'
# assign the /status endpoint to another variable called status_url (1 line)
status_url = f"{root_url}/status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(root_url, auth=('user', 'pass'))

# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code==200:
    print("reqeuest is ok")
else:
    print("reqeuest  not ok :{req.status_code}")


reqeuest is ok


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [112]:
# Set the countries_url variable (1 line)
countries_url = 'https://country-leaders.onrender.com/countries'

# Query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(countries_url)

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()

# Display the request's status code and the countries variable (1 line)
print(f"Status code: {req.status_code}, Countries: {countries}")


Status code: 403, Countries: {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [113]:
# Set the cookie_url variable (1 line)
cookie_url='https://country-leaders.onrender.com/cookie'
# Query the enpoint, set the cookies variable and display it (2 lines)
cookies=requests.get(cookie_url).cookies.get_dict()
print(cookies)


{'user_cookie': '5f6cc16c-b5c9-4f7e-b5a8-d555171263fd'}


Try to query the countries endpoint using the cookie, save the output and print it.

In [114]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries= requests.get(countries_url, cookies=cookies).json()


# display the countries variable (1 line)
print(countries)


['us', 'be', 'ma', 'ru', 'fr']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [115]:

# Set the leaders_url variable (1 line)
leaders_url='https://country-leaders.onrender.com/leaders'
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders=requests.get(leaders_url,cookies=cookies).json()

# display the leaders variable (1 line)
print(leaders)


{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [123]:
# Set the cookie_url variable (1 line)
cookie_url='https://country-leaders.onrender.com/cookie'
# Query the enpoint, set the cookies variable and display it (2 lines)
cookies=requests.get(cookie_url).cookies.get_dict()
print(cookies)

# Set the countries_url variable (1 line)
countries_url = 'https://country-leaders.onrender.com/countries'

# Query the /countries endpoint using the get() method and include the cookies (1 line)
countries = requests.get(countries_url, cookies=cookies).json()

# Display the countries to check if 'fr' or the country of your choice exists (1 line)
print(countries)


{'user_cookie': 'a2d64208-a127-4d54-8e49-52d37fe3e80e'}
['us', 'be', 'ma', 'ru', 'fr']


In [124]:

# Query the /leaders endpoint using cookies and parameters (take any country in countries)
leaders_url = 'https://country-leaders.onrender.com/leaders'  # Example for France
d={"country":'fr'}
# Assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies,params=d).json()

# Display the leaders variable (1 line)
print(leaders)
'''for i in leaders:
    if type(i)!=dict:
        print(i)'''

[{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_url': 'h

'for i in leaders:\n    if type(i)!=dict:\n        print(i)'

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [125]:
# 4 lines
leaders_per_country = {}
for country in countries:
    params={"country": country} 
    response = requests.get(f'https://country-leaders.onrender.com/leaders', cookies=cookies,params=params)
    leaders_per_country[country] = response.json() 
print(leaders_per_country)



{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [126]:
# < 15 lines
def get_leaders():
    #define the urls
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    countries_url = 'https://country-leaders.onrender.com/countries'
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    #get the cookies
    cookies = requests.get(cookie_url).cookies.get_dict()
    #get the countries
    countries = requests.get(countries_url, cookies=cookies).json()
    #loop over them and save their leaders in a dictionary
    leaders_per_country = {country: requests.get('https://country-leaders.onrender.com/leaders', cookies=cookies, params={'country': country}).json() for country in countries}
    #return the dictionary
    return leaders_per_country


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [127]:
# 2 lines
# Test the function and save the result
leaders_per_country = get_leaders()

# Print the result to check the output
print(leaders_per_country)



{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [128]:
import requests
from bs4 import BeautifulSoup

country = "us"
leader_name = leaders_per_country[country]
wikipedia_url = "https://fr.wikipedia.org/wiki/George_Washington" 

response = requests.get(wikipedia_url)

soup = BeautifulSoup(response.content, 'html.parser')

    
text = soup.get_text()

    
print(text)







George Washington — Wikipédia




































Aller au contenu







Menu principal





Menu principal
déplacer vers la barre latérale
masquer



		Navigation
	


AccueilPortails thématiquesArticle au hasardContact





		Contribuer
	


Débuter sur WikipédiaAideCommunautéModifications récentes



















Rechercher











Rechercher






















Apparence
















Faire un don

Créer un compte

Se connecter








Outils personnels





Faire un don Créer un compte Se connecter





		Pages pour les contributeurs déconnectés en savoir plus



ContributionsDiscussion




























Sommaire
déplacer vers la barre latérale
masquer




Début





1
Situation personnelle




Afficher / masquer la sous-section Situation personnelle





1.1
Origines familiales








1.2
Jeunesse et éducation










2
Carrière militaire








3
Mariage et vie à Mount Vernon








4
Guerre d'indépendance








5
Président des États

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [129]:
# 3 lines
pretty_html = soup.prettify()

print(pretty_html)


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   George Washington — Wikipédia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [130]:
paragraphs = []

    # Find all the <p> tags and iterate over them
for p in soup.find_all('p'):
    paragraphs.append(p.get_text())
print(paragraphs)

['Vous lisez un «\xa0bon article\xa0» labellisé en 2007.\n', 'Pour les articles homonymes, voir George Washington (homonymie).\n', '\n', "George Washington  (prononcé en anglais\xa0:\xa0/d͡ʒɔɹd͡ʒ ˈwɑʃɪŋtən/[note 1]), né le 22 février 1732[note 2] à Pope's Creek (colonie de Virginie) et mort le 14 décembre 1799 à Mount Vernon (État de Virginie), est un militaire, planteur esclavagiste et homme d'État américain, chef d’état-major de l’Armée continentale pendant la guerre d’indépendance entre 1775 et 1783 et premier président des États-Unis, en fonction de 1789 à 1797.\n", "Washington est l'un des planteurs les plus riches de Virginie avec son domaine de Mount Vernon. Grâce à sa participation à la guerre de Sept Ans qui se déroule entre 1756 et 1763, il devient rapidement célèbre des deux côtés de l'Atlantique et s'intéresse aux questions politiques. Son engagement dans la révolution américaine ainsi que sa réputation le portent au poste de commandant des troupes américaines, qu'il organi

In [167]:
# 2 lines
# Get all paragraph tags
paragraphs = [p.get_text() for p in soup.find_all('p')]
print(paragraphs)


['Vous lisez un «\xa0bon article\xa0» labellisé en 2007.\n', 'Pour les articles homonymes, voir George Washington (homonymie).\n', '\n', "George Washington  (prononcé en anglais\xa0:\xa0/d͡ʒɔɹd͡ʒ ˈwɑʃɪŋtən/[note 1]), né le 22 février 1732[note 2] à Pope's Creek (colonie de Virginie) et mort le 14 décembre 1799 à Mount Vernon (État de Virginie), est un militaire, planteur esclavagiste et homme d'État américain, chef d’état-major de l’Armée continentale pendant la guerre d’indépendance entre 1775 et 1783 et premier président des États-Unis, en fonction de 1789 à 1797.\n", "Washington est l'un des planteurs les plus riches de Virginie avec son domaine de Mount Vernon. Grâce à sa participation à la guerre de Sept Ans qui se déroule entre 1756 et 1763, il devient rapidement célèbre des deux côtés de l'Atlantique et s'intéresse aux questions politiques. Son engagement dans la révolution américaine ainsi que sa réputation le portent au poste de commandant des troupes américaines, qu'il organi

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [166]:
content_div = soup.find('div', {'class': 'mw-parser-output'})
first_paragraph = None

if content_div:
        paragraphs = content_div.find_all('p', recursive=False)  
        for paragraph in paragraphs:
            paragraph_text = paragraph.get_text(strip=True)  
            if paragraph_text:  
                first_paragraph = paragraph_text
                break  
print(first_paragraph or "No introductory paragraph found.")

George Washington(prononcé enanglais:/d͡ʒɔɹd͡ʒ ˈwɑʃɪŋtən/[note 1]), né le22 février 1732[note 2]àPope's Creek(colonie de Virginie) et mort le14 décembre 1799àMount Vernon(État de Virginie), est unmilitaire,planteur esclavagisteethomme d'Étataméricain, chef d’état-major de l’Armée continentalependant laguerre d’indépendanceentre 1775 et 1783 et premierprésident des États-Unis, en fonction de 1789 à 1797.


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [191]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    response = requests.get(wikipedia_url)

    soup = BeautifulSoup(response.content, 'html.parser')
 
    print(wikipedia_url) 
    content_div = soup.find('div', {'class': 'mw-parser-output'})
    first_paragraph = ''
    if content_div:
        paragraphs = content_div.find_all('p', recursive=False)  
        for paragraph in paragraphs:
            paragraph_text = paragraph.get_text(strip=True)  
            if paragraph_text:  
                first_paragraph = paragraph_text
                break  
    return first_paragraph

In [192]:
# Test: 3 lines
first_paragraph = get_first_paragraph("https://fr.wikipedia.org/wiki/George_Washington" )
#first_paragraph = get_first_paragraph("https://en.wikipedia.org/wiki/Thomas_Jefferson")

# Print the result to check the output
print(first_paragraph)

https://fr.wikipedia.org/wiki/George_Washington
George Washington(prononcé enanglais:/d͡ʒɔɹd͡ʒ ˈwɑʃɪŋtən/[note 1]), né le22 février 1732[note 2]àPope's Creek(colonie de Virginie) et mort le14 décembre 1799àMount Vernon(État de Virginie), est unmilitaire,planteur esclavagisteethomme d'Étataméricain, chef d’état-major de l’Armée continentalependant laguerre d’indépendanceentre 1775 et 1783 et premierprésident des États-Unis, en fonction de 1789 à 1797.


### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [193]:
# 3 lines
import re
help(re.sub)

# French
print(get_first_paragraph("https://fr.wikipedia.org/wiki/Albert_Einstein"))

# Spanish
print(get_first_paragraph("https://es.wikipedia.org/wiki/Albert_Einstein"))

# Japanese
print(get_first_paragraph("https://ja.wikipedia.org/wiki/アルベルト・アインシュタイン"))

# Arabic
print(get_first_paragraph("https://ar.wikipedia.org/wiki/ألبرت_أينشتاين"))




Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used.

https://fr.wikipedia.org/wiki/Albert_Einstein

https://es.wikipedia.org/wiki/Albert_Einstein
Albert Einstein(pronunciación en alemán:/ˈalbɐt ˈaɪnʃtaɪn/ⓘ;Ulm,Imperio alemán, 14 de marzo de 1879-Princeton,Estados Unidos, 18 de abril de 1955) fue unfísicoalemánde origenjudío, nacionalizado despuéssuizo,austriacoyestadounidense. Se le considera elcientíficomás importante, conocido y popular del sigloXX.[2]​[3]​
https://ja.wikipedia.org/wiki/アルベルト・アインシュタイン
アルベルト・アインシュタイン[注釈 1]（独:Albert Einstein[注釈 2][注釈 3][1][2]、1879年3月14日-1955年4月18日）は、ドイツ生まれの理論物理学者、社会主義者[3]。ユダヤ人。スイ

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [36]:
# 10 lines
import requests
from bs4 import BeautifulSoup
import re


def get_first_paragraph(wikipedia_url):
    patt=  r"<(?:b|strong|i)[^>]*>.*?<\/(?:b|strong|i)>"
    response = requests.get(wikipedia_url)

    soup = BeautifulSoup(response.content, 'html.parser')
 
    print(wikipedia_url) 
    #content_div = soup.find('div', {'class': 'mw-parser-output'})
    first_paragraph = ''
    #if content_div:
    paragraphs = soup.find_all('p')  
    for paragraph in paragraphs:
            paragraph_text = paragraph.get_text(strip=True)  
            cleaned_paragraph = re.sub(r'\[.*?\]', '', paragraph_text)
            if cleaned_paragraph and re.search(patt,str(paragraph) ):
                first_paragraph = cleaned_paragraph
                break  
    return first_paragraph

In [13]:
# Test with a Wikipedia URL
first_paragraph = get_first_paragraph("https://fr.wikipedia.org/wiki/George_Washington" )
print(first_paragraph)

https://fr.wikipedia.org/wiki/George_Washington
George Washington(prononcé enanglais:/d͡ʒɔɹd͡ʒ ˈwɑʃɪŋtən/), né le22 février 1732àPope's Creek(colonie de Virginie) et mort le14 décembre 1799àMount Vernon(État de Virginie), est unmilitaire,planteur esclavagisteethomme d'Étataméricain, chef d’état-major de l’Armée continentalependant laguerre d’indépendanceentre 1775 et 1783 et premierprésident des États-Unis, en fonction de 1789 à 1797.


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines
"""def get_first_paragraph(wikipedia_url):
    response = requests.get(wikipedia_url)

    soup = BeautifulSoup(response.content, 'html.parser')
 
    print(wikipedia_url) 
    content_div = soup.find('div', {'class': 'mw-parser-output'})
    first_paragraph = ''
    if content_div:
        paragraphs = content_div.find_all('p', recursive=False)  
        for paragraph in paragraphs:
            paragraph_text = paragraph.get_text(strip=True)  
            cleaned_paragraph = re.sub(r'^\W+|\W+$', '', 
    re.sub(r'\.{2,}', '.', 
        re.sub(r'[^\w\s.,!?\'"-]', '', 
            re.sub(r'\s+', ' ', 
                re.sub(r'<.*?>', '', 
                    re.sub(r'\[[^\]]*\]', '', 
                        re.sub(r'\(.*?\)', '', 
                            re.sub(r'\[.*?\]', '', paragraph_text))))))))
            if cleaned_paragraph :
                first_paragraph = cleaned_paragraph
                break  
    return first_paragraph"""



https://fr.wikipedia.org/wiki/George_Washington
George Washington, né le22 février 1732àPope's Creek et mort le14 décembre 1799àMount Vernon, est unmilitaire,planteur esclavagisteethomme d'Étataméricain, chef détat-major de lArmée continentalependant laguerre dindépendanceentre 1775 et 1783 et premierprésident des États-Unis, en fonction de 1789 à 1797


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [14]:
def get_leaders():
    #define the urls
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    countries_url = 'https://country-leaders.onrender.com/countries'
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    #get the cookies
    cookies = requests.get(cookie_url).cookies.get_dict()
    #get the countries
    countries = requests.get(countries_url, cookies=cookies).json()
    #loop over them and save their leaders in a dictionary
    leaders_per_country = {}
    for country in countries:
        response = requests.get(leaders_url, cookies=cookies, params={'country': country})
        leaders = response.json()
        
        leaders_per_country[country] = {}
        
        # Loop over the leaders and get their first paragraph
        for leader_info in leaders:
            if type(leader_info)==dict:
                leader_name = f"{leader_info.get('first_name')} {leader_info.get('last_name')}"
                wikipedia_url = leader_info.get('wikipedia_url')
                if wikipedia_url:
                    first_paragraph = get_first_paragraph(wikipedia_url)
                    print(first_paragraph)
                    leaders_per_country[country][leader_name] = first_paragraph
                    print(f"Leader: {leader_name}, Country: {country}, First Paragraph: {first_paragraph}")
    return leaders_per_country 

In [15]:
# Check the output of your function (2 lines)
get_leaders()

https://en.wikipedia.org/wiki/George_Washington
George Washington(February 22, 1732 –December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAmerican Revolutionary Warand then served as president of theConstitutional Conventionin 1787, which drafted the currentConstitution of the United States. Washington has thus become commonly known as the "Father of his Country".
Leader: George Washington, Country: us, First Paragraph: George Washington(February 22, 1732 –December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAm

{'us': {'George Washington': 'George Washington(February 22, 1732\xa0–December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAmerican Revolutionary Warand then served as president of theConstitutional Conventionin 1787, which drafted the currentConstitution of the United States. Washington has thus become commonly known as the "Father of his Country".',
  'Barack Obama': 'Barack Hussein Obama II(born August 4, 1961) is an American politician who served as the 44thpresident of the United Statesfrom 2009 to 2017. As a member of theDemocratic Party, he was the firstAfrican-Americanpresident in U.S. history. Obama previously served as aU.S. senatorrepresentingIllinoisfrom 2005 to 2008 and as anIllinois state senatorfrom 1997 to 2004.',
  'Abraham Lincoln

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [25]:
import requests
from bs4 import BeautifulSoup

def get_new_cookies(cookie_url):
    cookies = requests.get(cookie_url).cookies.get_dict()
    return cookies


def get_leaders():
    # Define the URLs
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    countries_url = 'https://country-leaders.onrender.com/countries'
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    
    # Get the cookies
    cookies = get_new_cookies(cookie_url)
    
    # Get the countries
    countries = requests.get(countries_url, cookies=cookies).json()
    
    # Loop over countries and get the leaders
    leaders_per_country = {}
    for country in countries:
        response = requests.get(leaders_url, cookies=cookies, params={'country': country})
        leaders = response.json()
        
        # Store the leaders' first paragraphs
        leaders_per_country[country] = {}
        for leader_info in leaders:
            if type(leader_info)==dict:
            # Combine first and last names to get the full name
                leader_name = f"{leader_info.get('first_name')} {leader_info.get('last_name')}"
                wikipedia_url = leader_info.get('wikipedia_url')
                if wikipedia_url:
                    print(wikipedia_url)
                    first_paragraph = get_first_paragraph(wikipedia_url)
                    leaders_per_country[country][leader_name] = first_paragraph
                    print(f"Leader: {leader_name}, Country: {country}, First Paragraph: {first_paragraph}")
    
    return leaders_per_country



Check the output of your function again.

In [134]:
# Check the output of your function (1 line)
leaders_info = get_leaders()
print(leaders_info)



https://en.wikipedia.org/wiki/George_Washington
Leader: George Washington, Country: us, First Paragraph: 
https://en.wikipedia.org/wiki/Barack_Obama
Leader: Barack Obama, Country: us, First Paragraph: 
https://en.wikipedia.org/wiki/Abraham_Lincoln
Leader: Abraham Lincoln, Country: us, First Paragraph: 
https://en.wikipedia.org/wiki/George_W._Bush
Leader: George Bush, Country: us, First Paragraph: 
https://en.wikipedia.org/wiki/Bill_Clinton
Leader: Bill Clinton, Country: us, First Paragraph: 
https://en.wikipedia.org/wiki/Joe_Biden
Leader: Joe Biden, Country: us, First Paragraph: 
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
Leader: Franklin Roosevelt, Country: us, First Paragraph: 
https://en.wikipedia.org/wiki/Andrew_Johnson
Leader: Andrew Johnson, Country: us, First Paragraph: 
https://en.wikipedia.org/wiki/Gerald_Ford
Leader: Gerald Ford, Country: us, First Paragraph: 
https://en.wikipedia.org/wiki/Richard_Nixon
Leader: Richard Nixon, Country: us, First Paragraph: 
https://en

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [37]:
# < 20 lines
import requests
from bs4 import BeautifulSoup
import re


def get_first_paragraph(wikipedia_url,session):
    patt=  r"<(?:b|strong|i)[^>]*>.*?<\/(?:b|strong|i)>"
    session = requests.Session()
    response = session.get(wikipedia_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
 
    print(wikipedia_url) 
    #content_div = soup.find('div', {'class': 'mw-parser-output'})
    first_paragraph = ''
    #if content_div:
    paragraphs = soup.find_all('p')  
    for paragraph in paragraphs:
            paragraph_text = paragraph.get_text(strip=True)  
            cleaned_paragraph = re.sub(r'\[.*?\]', '', paragraph_text)
            if cleaned_paragraph and re.search(patt,str(paragraph) ):
                first_paragraph = cleaned_paragraph
                break  
    return first_paragraph



Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [33]:
# <25 lines
import re
def get_leaders():
    # Define the URLs
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    countries_url = 'https://country-leaders.onrender.com/countries'
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    
    # Get the cookies
    cookies = get_new_cookies(cookie_url)
    
    # Get the countries
    countries = requests.get(countries_url, cookies=cookies).json()
    session = requests.Session()
    # Loop over countries and get the leaders
    leaders_per_country = {}
    for country in countries:
        response = requests.get(leaders_url, cookies=cookies, params={'country': country})
        leaders = response.json()
        # Store the leaders' first paragraphs
        leaders_per_country[country] = {}
        cookies = get_new_cookies(cookie_url)
        for leader_info in leaders:
            if type(leader_info)==dict:
            # Combine first and last names to get the full name
                leader_name = f"{leader_info.get('first_name')} {leader_info.get('last_name')}"
                wikipedia_url = leader_info.get('wikipedia_url')
                if wikipedia_url:
                    first_paragraph = get_first_paragraph(wikipedia_url,session)
                    print(first_paragraph)
                    leaders_per_country[country][leader_name] = first_paragraph
                    print(f"Leader: {leader_name}, Country: {country}, First Paragraph: {first_paragraph}")
    
    return leaders_per_country




In [ ]:
# <25 lines
'''import requests

def get_leaders():
    # Define the URLs
    leaders_url = 'https://country-leaders.onrender.com/leaders'
    countries_url = 'https://country-leaders.onrender.com/countries'
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    
    # Get the cookies (assuming the function `get_new_cookies()` is defined elsewhere)
    cookies = get_new_cookies(cookie_url)
    
    # Get the list of countries
    countries = requests.get(countries_url, cookies=cookies).json()
    session = requests.Session()
    
    # Dictionary to store leaders' first paragraphs
    leaders_per_country = {}

    # Loop over countries and get the leaders
    for country in countries:
        response = requests.get(leaders_url, cookies=cookies, params={'country': country})
        leaders = response.json()
        
        # Initialize a dictionary to store leaders' first paragraphs for the country
        leaders_per_country[country] = {}

        # Loop through each leader's info
        for leader_info in leaders:
            if isinstance(leader_info, dict):  # Check if leader_info is a dictionary
                # Combine first and last names to get the full name
                leader_name = f"{leader_info.get('first_name', '')} {leader_info.get('last_name', '')}".strip()
                wikipedia_url = leader_info.get('wikipedia_url')

                # Proceed only if there is a Wikipedia URL
                if wikipedia_url:
                    # Fetch the first paragraph from Wikipedia page
                    first_paragraph = get_first_paragraph(wikipedia_url)
                    
                    # Print the first paragraph for debugging
                    print(f"Leader: {leader_name}, Country: {country}, First Paragraph: {first_paragraph}")
                    
                    # Store the first paragraph for the leader in the dictionary
                    leaders_per_country[country][leader_name] = first_paragraph

    return leaders_per_country'''


Test your new functions.



In [34]:
get_leaders()

https://en.wikipedia.org/wiki/George_Washington
George Washington(February 22, 1732 –December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAmerican Revolutionary Warand then served as president of theConstitutional Conventionin 1787, which drafted the currentConstitution of the United States. Washington has thus become commonly known as the "Father of his Country".
Leader: George Washington, Country: us, First Paragraph: George Washington(February 22, 1732 –December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAm

{'us': {'George Washington': 'George Washington(February 22, 1732\xa0–December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAmerican Revolutionary Warand then served as president of theConstitutional Conventionin 1787, which drafted the currentConstitution of the United States. Washington has thus become commonly known as the "Father of his Country".',
  'Barack Obama': 'Barack Hussein Obama II(born August 4, 1961) is an American politician who served as the 44thpresident of the United Statesfrom 2009 to 2017. As a member of theDemocratic Party, he was the firstAfrican-Americanpresident in U.S. history. Obama previously served as aU.S. senatorrepresentingIllinoisfrom 2005 to 2008 and as anIllinois state senatorfrom 1997 to 2004.',
  'Abraham Lincoln

In [32]:
get_leaders()

https://en.wikipedia.org/wiki/George_Washington
George Washington(February 22, 1732 –December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAmerican Revolutionary Warand then served as president of theConstitutional Conventionin 1787, which drafted the currentConstitution of the United States. Washington has thus become commonly known as the "Father of his Country".
Leader: George Washington, Country: us, First Paragraph: George Washington(February 22, 1732 –December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAm

{'us': {'George Washington': 'George Washington(February 22, 1732\xa0–December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAmerican Revolutionary Warand then served as president of theConstitutional Conventionin 1787, which drafted the currentConstitution of the United States. Washington has thus become commonly known as the "Father of his Country".',
  'Barack Obama': 'Barack Hussein Obama II(born August 4, 1961) is an American politician who served as the 44thpresident of the United Statesfrom 2009 to 2017. As a member of theDemocratic Party, he was the firstAfrican-Americanpresident in U.S. history. Obama previously served as aU.S. senatorrepresentingIllinoisfrom 2005 to 2008 and as anIllinois state senatorfrom 1997 to 2004.',
  'Abraham Lincoln

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [35]:
# 3 lines
import json
leaders_per_country = get_leaders()
with open('leaders.json', 'w', encoding='utf-8') as f:
    json.dump(leaders_per_country, f, ensure_ascii=False, indent=4)

https://en.wikipedia.org/wiki/George_Washington
George Washington(February 22, 1732 –December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAmerican Revolutionary Warand then served as president of theConstitutional Conventionin 1787, which drafted the currentConstitution of the United States. Washington has thus become commonly known as the "Father of his Country".
Leader: George Washington, Country: us, First Paragraph: George Washington(February 22, 1732 –December 14, 1799) was aFounding Father of the United States, military officer, and farmer who served as the firstpresident of the United Statesfrom 1789 to 1797. Appointed by theSecond Continental Congressas commander of theContinental Armyin 1775, Washington ledPatriotforces to victory in theAm

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [212]:
# 3 lines
def read_leaders_from_file(filename):
    with open(filename, 'r') as f:
        leaders_per_country = json.load(f)
    return leaders_per_country

leaders_data = read_leaders_from_file('leaders.json')
print(leaders_data)

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

Make a function `save(leaders_per_country)` to call this code easily.

In [211]:
# 3 lines
import json

def save(leaders_per_country, filename):
    try:
        with open(filename, 'w') as f:
            json.dump(leaders_per_country, f, indent=4)
        print(f"Data successfully saved to {filename}")
    except Exception as e:
        print(f"Error saving data to {filename}: {e}")


In [210]:
# Call the function (1 line)
save(leaders_per_country, 'leaders.json')


Data successfully saved to leaders.json


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!